# Стохастический градиентный и координатный спуски

Для каждого задания указано количество баллов (если они оцениваются отдельно) + 1 балл за аккуратное и полное выполнение всего задания

## Загрузка и подготовка данных

**Загрузите уже знакомый вам файл *Advertising.csv* как объект DataFrame.** 

In [10]:
import pandas as pd
import numpy as np

In [46]:
df = pd.read_csv('data/Advertising.csv',index_col=0)
df.head() # посмотрим на корректность загрузки

,TV,radio,newspaper,sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


**Проверьте, есть ли в данных пропуски и, если они есть - удалите их**

In [47]:
cols_null_percent = df.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null) # отобразим количество пропусков в процентах

Series([], dtype: float64)

В данных нет пропусков

**Преобразуйте ваши признаки в массивы NumPy и разделите их на переменные X (предикторы) и y(целевая переменная)** 

In [56]:
X = np.array(df[['TV', 'radio', 'newspaper']]) # предикаторы
y = np.array(df['sales']) # целевая переменная

## Координатный спуск (3 балла)

**Добавим единичный столбец для того, чтобы у нас был свободный коэффициент в уравнении регрессии:**

In [57]:
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])

**Нормализуем данные: обычно это необходимо для корректной работы алгоритма**

In [58]:
X = X / np.sqrt(np.sum(np.square(X), axis=0))

**Реализуйте алгоритм координатного спуска:** (3 балла)

Ниже приведен алгоритм:

<a href="https://ibb.co/Th3BQFn"><img src="https://i.ibb.co/DK2DBS6/zascas.jpg" alt="zascas" border="0"></a>

Примечание: 1000 итераций здесь указаны для этого задания, на самом деле их может быть намного больше, нет детерменированного значения.

Вам необходимо реализовать координатный спуск, и вывести веса в модели линейной регрессии.

In [60]:
w = np.zeros(X.shape[1])

for i in range(1000):
    r = y - X.dot(w)
    for j in range(len(w)):
        r = r + X[:, j] * w[j]
        w[j] = X[:, j].dot(r)
        r = r - X[:, j] * w[j]
print('Веса модели: ', w)

Веса модели:  [ 41.56217205 110.13144155  73.52860638  -0.55006384]


Сравните результаты с реализацией линейной регрессии из библиотеки sklearn:

In [21]:
from sklearn.linear_model import LinearRegression
 
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
 
print(model.coef_)

[ 41.56217205 110.13144155  73.52860638  -0.55006384]


Веса моделей полученные из библиотеки sklearn и при помощи ручной реализации совпадают. 

Если вы все сделали верно, они должны практически совпасть!

## Стохастический градиентный спуск (6 баллов)

**Отмасштабируйте столбцы исходной матрицы *X* (которую мы не нормализовали еще!). Для того, чтобы это сделать, надо вычесть из каждого значения среднее и разделить на стандартное отклонение** (0.5 баллов)

In [27]:
# дублируем определение X, для того чтобы обнулить предыдущую нормализацию
X = np.array(df[['TV', 'radio', 'newspaper']]) 
X = (X - np.mean(X))/np.std(X)

**Добавим единичный столбец**

In [28]:
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])

**Создайте функцию mse_error для вычисления среднеквадратичной ошибки, принимающую два аргумента: реальные значения и предсказывающие, и возвращающую значение mse** (0.5 балла)

In [37]:
# Функция для расчета среднеквадратичной ошибки
def mse_error(y, y_pred):
    MSE = np.mean((y.T-y_pred)**2)
    return MSE

**Сделайте наивный прогноз: предскажите продажи средним значением. После этого рассчитайте среднеквадратичную ошибку для этого прогноза** (0.5 балла)

In [38]:
y_pred = np.ones(y.shape[0])*np.mean(y)
print(mse_error(y, y_pred))

27.085743750000002


**Создайте функцию *lin_pred*, которая может по матрице предикторов *X* и вектору весов линейной модели *w* получить вектор прогнозов** (0.5 балла)

In [64]:
# Функция предсказания
def lin_pred(X, w):
    y_pred = X @ w
    return y_pred

**Создайте функцию *stoch_grad_step* для реализации шага стохастического градиентного спуска. (1.5 балла) 
Функция должна принимать на вход следующие аргументы:**
* матрицу *X*
* вектора *y* и *w*
* число *train_ind* - индекс объекта обучающей выборки (строки матрицы *X*), по которому считается изменение весов
* число *$\eta$* (eta) - шаг градиентного спуска

Результатом будет вектор обновленных весов

Шаг для стохастического градиентного спуска выглядит следующим образом:

$$\Large w_j \leftarrow w_j - \frac{2\eta}{\ell} \sum_{i=1}^\ell{{x_{ij}((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}}$$

Для того, чтобы написать функцию, нужно сделать следующее:
    
*  посчитать направление изменения: умножить объект обучающей выборки на 2 и на разницу между предсказанным значением и реальным, а потом поделить на количество элементов в выборке.
* вернуть разницу между вектором весов и направлением изменения, умноженным на шаг градиентного спуска

In [73]:
def stoch_grad_step(X, y, w, train_ind, eta):
    y_pred = lin_pred(X, w)
    a = (2 * X[train_ind] * (y_pred[train_ind] - y[train_ind]))/X.size
    b = (w - a)*eta
    return b

**Создайте функцию *stochastic_gradient_descent*, для реализации стохастического градиентного спуска (2.5 балла)**

**Функция принимает на вход следующие аргументы:**
- Матрицу признаков X
- Целевую переменнную
- Изначальную точку (веса модели)
- Параметр, определяющий темп обучения
- Максимальное число итераций
- Евклидово расстояние между векторами весов на соседних итерациях градиентного спуска,при котором алгоритм прекращает работу 

**На каждой итерации в вектор (список) должно записываться текущее значение среднеквадратичной ошибки. Функция должна возвращать вектор весов $w$, а также вектор (список) ошибок.**

Алгоритм сследующий:
    
* Инициализируйте расстояние между векторами весов на соседних итерациях большим числом (можно бесконечностью)
* Создайте пустой список для фиксации ошибок
* Создайте счетчик итераций
* Реализуйте основной цикл обучения пока расстояние между векторами весов больше того, при котором надо прекратить работу (когда расстояния станут слишком маленькими - значит, мы застряли в одном месте) и количество итераций меньше максимально разрешенного: сгенерируйте случайный индекс, запишите текущую ошибку в вектор ошибок, запишите в переменную текущий шаг стохастического спуска с использованием функции, написанной ранее. Далее рассчитайте текущее расстояние между векторами весов и прибавьте к счетчику итераций 1.
* Верните вектор весов и вектор ошибок

In [248]:
def stochastic_gradient_descent(X, y, w, learn_rate, iter, eps):
    dist=1e100
    errors = []
    i = 0
    while True:
        print('Weight: ', w)
        print('Iter: ', i)
        print('dist: ', dist)
        if dist <= eps:
            break
        else:
            rand_index = np.random.randint(0, y.shape[0])
            y_pred = lin_pred(X, w)
            #grad = np.dot(X[rand_index], (y_pred[rand_index] - y[rand_index]))/y.shape[0]
            w_new = w - learn_rate * np.gradient(X, y, w)
            errors.append(mse_error(y, y_pred))
            #learn_rate = stoch_grad_step(X, y, w, rand_index, learn_rate)
            dist = np.linalg.norm(w - w_new)
            w = w_new
            
        i += 1
        if i > iter:
            break
    
    return w, errors
            

 **Запустите $10^5$ итераций стохастического градиентного спуска. Укажите вектор начальных весов, состоящий из нулей. Можете поэкспериментировать с параметром, отвечающим за темп обучения.**

**Постройте график зависимости ошибки от номера итерации**

In [208]:
np.dot(X.T, (y_pred - y))/y.shape[0]

array([ 7.99360578e-17, -1.44875118e-01, -1.13875288e-01, -4.86842464e-02])

In [212]:
np.dot(X[7], (y_pred[7] - y[7]))/y.shape[0]

array([2.90797664e-04, 2.05413582e-04, 2.06674147e-04, 8.99779296e-05])

In [170]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
print('w_hat_clean_sklearn:', model.coef_.round(4))
new_prediction = model.predict(X)
mse_error(y, new_prediction)

w_hat_clean_sklearn: [ 41.5622 110.1314  73.5286  -0.5501]


2.784126314510936

In [181]:
w0 = np.zeros(X.shape[1])
stochastic_gradient_descent(X, y, w0, 10, 100000, 0.001)

Weight:  [0. 0. 0. 0.]
Iter:  0
dist:  1e+100
Weight:  [0. 0. 0. 0.]
Iter:  1
dist:  0.0


(array([0., 0., 0., 0.]), [223.71625])

**Выведите вектор весов, к которому сошелся метод.**

In [ ]:
# ваш код

**Выведите среднеквадратичную ошибку на последней итерации.**

In [ ]:
# ваш код